In [15]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [16]:
with open('/kaggle/input/hamlet-txt/hamlet_TXT_FolgerShakespeare.txt','r') as file:
    text=file.read().lower()


In [17]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4751

In [18]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'you': 5,
 'a': 6,
 'i': 7,
 'my': 8,
 'hamlet': 9,
 'in': 10,
 'it': 11,
 'that': 12,
 'is': 13,
 'not': 14,
 'his': 15,
 'this': 16,
 'with': 17,
 'but': 18,
 'he': 19,
 'for': 20,
 'your': 21,
 'me': 22,
 'as': 23,
 'be': 24,
 'lord': 25,
 'what': 26,
 'king': 27,
 'him': 28,
 'so': 29,
 'have': 30,
 'will': 31,
 'horatio': 32,
 'do': 33,
 'no': 34,
 'we': 35,
 'on': 36,
 'are': 37,
 'all': 38,
 'queen': 39,
 'by': 40,
 'our': 41,
 'they': 42,
 'polonius': 43,
 'shall': 44,
 'if': 45,
 'or': 46,
 'laertes': 47,
 'o': 48,
 'good': 49,
 'thou': 50,
 'come': 51,
 'now': 52,
 'from': 53,
 'more': 54,
 'let': 55,
 'her': 56,
 'ophelia': 57,
 'how': 58,
 'was': 59,
 'thy': 60,
 "'t": 61,
 'at': 62,
 'like': 63,
 'most': 64,
 'would': 65,
 'there': 66,
 'them': 67,
 'sir': 68,
 'well': 69,
 'rosencrantz': 70,
 'know': 71,
 'enter': 72,
 "'tis": 73,
 'may': 74,
 "th'": 75,
 'us': 76,
 'go': 77,
 'love': 78,
 'then': 79,
 'did': 80,
 'hath': 81,
 'v

In [19]:
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [20]:
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

20

In [21]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,   40, 1944],
       [   0,    0,    0, ...,   40, 1944,  933],
       [   0,    0,    0, ...,    0, 1945,   40],
       ...,
       [   0,    0,    0, ...,    0, 1937,   37],
       [   0,    0,    0, ..., 1937,   37,  507],
       [   0,    0,    0, ...,   37,  507,  204]], dtype=int32)

In [22]:
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [23]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU

## Define the model
model=Sequential()
model.add(Embedding(total_words,100,input_shape=(max_sequence_len-1,)))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

# #Compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 19, 100)        │       475,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 19, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 19, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4751)           │       479,851 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,205,951 (4.60 MB)

 Trainable params: 1,205,951 (4.60 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
## GRU RNN
## Define the model
model2=Sequential()
model2.add(Embedding(input_dim=total_words,output_dim=100,input_shape=(max_sequence_len-1,)))
model2.add(GRU(150,return_sequences=True))
model2.add(Dropout(0.2))
model2.add(GRU(100))
model2.add(Dense(total_words,activation="softmax"))

# #Compile the model
model2.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model2.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 19, 100)        │       475,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 19, 150)        │       113,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 19, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 100)            │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4751)           │       479,851 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,143,951 (4.36 MB)

 Trainable params: 1,143,951 (4.36 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
## Train the model
history=model.fit(x_train,y_train,epochs=80,validation_data=(x_test,y_test),verbose=1,callbacks=[early_stopping])


Epoch 1/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.0307 - loss: 7.1077 - val_accuracy: 0.0336 - val_loss: 6.6829
Epoch 2/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0355 - loss: 6.4326 - val_accuracy: 0.0383 - val_loss: 6.7597
Epoch 3/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0447 - loss: 6.2609 - val_accuracy: 0.0485 - val_loss: 6.7880
Epoch 4/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0519 - loss: 6.1254 - val_accuracy: 0.0481 - val_loss: 6.7775
Epoch 5/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.0570 - loss: 5.9696 - val_accuracy: 0.0558 - val_loss: 6.8154
Epoch 6/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0670 - loss: 5.8466 - val_accuracy: 0.0594 - val_loss: 6.8363
Epoch 7/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0765 - loss: 5.6755 - val_accuracy: 0.0602 - val_loss: 6.8931
Epoch 8/80
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0841 - loss: 5.5330 - val_acc

In [29]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [35]:
input_text="Enter Barnardo and Francisco two"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:Enter Barnardo and Francisco two
Next Word PRediction:the


In [31]:
model.save("next_word_lstm.h5")
## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)